# 🚀 Getting Started with NeuralLayers

Welcome to NeuralLayers! This notebook will guide you through:
1. Basic setup and imports
2. Creating your first unified neural-logical network
3. Understanding the output
4. Monitoring system states

**Prerequisites:** Python 3.8+, PyTorch 2.0+

## 📦 Step 1: Imports and Setup

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

import torch
import numpy as np
import matplotlib.pyplot as plt

# Import NeuralLayers components
from logicalbrain_network import UnifiedBrainLogicNetwork
from utils import Config, Logger, DeviceManager

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

## ⚙️ Step 2: Configuration

NeuralLayers uses YAML configuration for easy customization.

In [ ]:
# Load configuration
config = Config('../config.yaml')

print("📊 Model Configuration:")
print(f"  Input dimension: {config.get('model.input_dim')}")
print(f"  Hidden dimension: {config.get('model.hidden_dim')}")
print(f"  Output dimension: {config.get('model.output_dim')}")

print("\n🧠 Brain Parameters:")
print(f"  Resting potential: {config.get('brain.V_rest')} mV")
print(f"  ATP baseline: {config.get('brain.ATP_baseline')} μM")

## 🖥️ Step 3: Device Setup

Automatically use GPU if available.

In [ ]:
device_manager = DeviceManager(config)
device = device_manager.device

print(f"🔧 Computing device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🧠 Step 4: Create the Unified Network

Initialize the complete neural-logical framework.

In [ ]:
# Create model
model = UnifiedBrainLogicNetwork(
    input_dim=config.get('model.input_dim'),
    hidden_dim=config.get('model.hidden_dim'),
    output_dim=config.get('model.output_dim')
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✨ Model initialized!")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: ~{total_params * 4 / 1e6:.1f} MB (FP32)")

## 🎯 Step 5: Forward Pass

Process input through the unified network.

In [ ]:
# Create sample input
batch_size = 8
input_dim = config.get('model.input_dim')

x = torch.randn(batch_size, input_dim).to(device)

# Forward pass
with torch.no_grad():
    output = model(x)

print("📤 Output keys:", list(output.keys()))
print(f"\n✅ Final output shape: {output['output'].shape}")
print(f"✅ Membrane potential shape: {output['membrane_potential'].shape}")
print(f"✅ Truth values shape: {output['truth_values'].shape}")

## 📊 Step 6: Examine System State

The system tracks 8 state variables.

In [ ]:
state = output['system_state']

print("🔬 System State Variables:\n")

state_descriptions = {
    'V': ('Membrane Potential', 'mV', (-70, 40)),
    'NT': ('Neurotransmitter', 'μM', (0, 10)),
    'Ca': ('Calcium', 'nM', (0, 1000)),
    'ATP': ('Energy (ATP)', 'μM', (1000, 10000)),
    'g': ('Glial State', '', (0, 1)),
    'Ψ': ('Cognitive State', '', None),
    'τ': ('Truth Values', '', (0, 1)),
    'ω': ('Reasoning Momentum', '', None)
}

for component, (name, unit, bounds) in state_descriptions.items():
    if component in state:
        mean_val = state[component].mean().item()
        std_val = state[component].std().item()
        
        unit_str = f" {unit}" if unit else ""
        bounds_str = f" (bounds: {bounds})" if bounds else ""
        
        print(f"  {component} - {name:20s}: {mean_val:8.2f} ± {std_val:6.2f}{unit_str}{bounds_str}")

## 📈 Step 7: Visualize State Evolution

Process multiple time steps and visualize.

In [ ]:
# Simulate 50 time steps
num_steps = 50
history = {'V': [], 'ATP': [], 'Ca': [], 'τ': []}

x = torch.randn(1, input_dim).to(device)

with torch.no_grad():
    for step in range(num_steps):
        output = model(x)
        state = output['system_state']
        
        # Track key variables
        history['V'].append(state['V'].mean().item())
        history['ATP'].append(state['ATP'].mean().item())
        history['Ca'].append(state['Ca'].mean().item())
        history['τ'].append(state['τ'].mean().item())

print(f"✅ Simulated {num_steps} time steps")

In [ ]:
# Plot evolution
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('System State Evolution', fontsize=16, fontweight='bold')

# Membrane potential
axes[0, 0].plot(history['V'], color='#2E86AB', linewidth=2)
axes[0, 0].axhline(y=-70, color='gray', linestyle='--', alpha=0.5, label='Rest')
axes[0, 0].set_title('Membrane Potential (V)', fontweight='bold')
axes[0, 0].set_ylabel('mV')
axes[0, 0].grid(alpha=0.3)
axes[0, 0].legend()

# ATP
axes[0, 1].plot(history['ATP'], color='#A23B72', linewidth=2)
axes[0, 1].axhline(y=5000, color='gray', linestyle='--', alpha=0.5, label='Baseline')
axes[0, 1].set_title('Energy (ATP)', fontweight='bold')
axes[0, 1].set_ylabel('μM')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].legend()

# Calcium
axes[1, 0].plot(history['Ca'], color='#F18F01', linewidth=2)
axes[1, 0].axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='Baseline')
axes[1, 0].set_title('Calcium (Ca)', fontweight='bold')
axes[1, 0].set_ylabel('nM')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].legend()

# Truth values
axes[1, 1].plot(history['τ'], color='#6A994E', linewidth=2)
axes[1, 1].set_title('Truth Values (τ)', fontweight='bold')
axes[1, 1].set_ylabel('Probability')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization complete!")

## 🎓 Summary

You've successfully:
1. ✅ Set up NeuralLayers with configuration
2. ✅ Created a unified neural-logical network
3. ✅ Performed forward passes
4. ✅ Examined system states
5. ✅ Visualized state evolution

### 🚀 Next Steps

- **Notebook 02:** Brain Network Basics - Deep dive into biophysical dynamics
- **Notebook 03:** Logical Reasoning - Explore truth valuation and inference
- **Notebook 04:** Consciousness Modeling - 7-layer consciousness hierarchy
- **Notebook 05:** Training & Optimization - Build real applications

### 📚 Resources

- [README](../README.md) - Complete documentation
- [Examples](../examples/) - Advanced tutorials
- [Tests](../tests/) - See how components are tested

---

**Happy neural-logical computing!** 🧠⚡